In [ ]:
# The accuracy of data without Macroeconomic variables


In [1]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
# 用於二元目標變數
from scipy.stats import pointbiserialr
from functools import reduce
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE

In [2]:
# Import GDP Data
from data import OverallGeneralEcon, df, MacroInfo, JuridicalInfo
from module1 import predict_for_month, find_best_threshold, predicted_labels, predicted_accuracy

c:\Users\user1\Desktop\repo2\revenue-prediction\data.py:48: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Dream_report = pd.read_sql(qr_sub, conn)
c:\Users\user1\Desktop\repo2\revenue-prediction\data.py:72: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  stock_info = pd.read_sql(stock_qr_sub, conn)
c:\Users\user1\Desktop\repo2\revenue-prediction\data.py:95: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  MacroInfo = pd.read_sql(macro_qr_sub, conn)


            年月     代號           數值                  名稱
0       202412  19600        1.060       經濟成長率(GDP)–單季
1       202412  19700   201644.000      國內生產毛額(GDP)–美元
2       202412  19710  6404219.000      國內生產毛額(GDP)–台幣
3       202412  19750     8567.000  平均每人國內生產毛額(GDP)–美元
4       202412  19760   272086.000  平均每人國內生產毛額(GDP)–台幣
...        ...    ...          ...                 ...
263615  201401  USA96      954.642              美國營建支出
263616  201401  USA97        0.000           美國新屋開工年增率
263617  201401  USA98     1723.614             美國企業存貨額
263618  201401  USA99        1.600          美國核心CPI年增率
263619  201401  ZAF00      100.300        OECD領先指標(南非)

[263620 rows x 4 columns]


c:\Users\user1\Desktop\repo2\revenue-prediction\data.py:107: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  JuridicalInfo = pd.read_sql(juridical_qr_sub, conn)


In [5]:
df2 = df.copy()
df2.columns

Index(['ddate', 'stockid', 'direction', 'rev', 'mom', 'yoy', 'revd1', 'yoyd1',
       'mond1', 'revd2', 'yoyd2', 'mond2', 'revd3', 'yoyd3', 'mond3', 'revd4',
       'yoyd4', 'mond4', 'revd5', 'yoyd5', 'mond5', 'revd6', 'yoyd6', 'mond6',
       'revd7', 'yoyd7', 'mond7', 'revd8', 'yoyd8', 'mond8', 'revd9', 'yoyd9',
       'mond9', 'revd10', 'yoyd10', 'mond10', 'revd11', 'yoyd11', 'mond11',
       'revd12', 'yoyd12', 'mond12'],
      dtype='object')

In [9]:
# df2.isna().sum() # no NAs in data

In [11]:
# Split data by TimeSeriesSplit
tscv_df2 = TimeSeriesSplit(n_splits=50)

# each split contains train set & test set 'index'
splits_df2 = list(tscv_df2.split(df2))

# 使用最後一組的 trained model index 切分 training & testing data
train_idx_df2 = splits_df2[-1][0]
test_idx_df2 = splits_df2[-1][1]

In [12]:
df2_var = ['revd1', 'yoyd1',
       'mond1', 'revd2', 'yoyd2', 'mond2', 'revd3', 'yoyd3', 'mond3', 'revd4',
       'yoyd4', 'mond4', 'revd5', 'yoyd5', 'mond5', 'revd6', 'yoyd6', 'mond6',
       'revd7', 'yoyd7', 'mond7', 'revd8', 'yoyd8', 'mond8', 'revd9', 'yoyd9',
       'mond9', 'revd10', 'yoyd10', 'mond10', 'revd11', 'yoyd11', 'mond11',
       'revd12', 'yoyd12', 'mond12']

df2_targ = ['direction']



In [14]:
# Prepare data for separating
y_df2 = df2[df2_targ]
X_df2 = df2[df2_var]

In [17]:
# Extract test and train data from overall data
X_train_df2, X_test_df2 = X_df2.iloc[train_idx_df2], X_df2.iloc[test_idx_df2]
y_train_df2, y_test_df2 = y_df2.iloc[train_idx_df2], y_df2.iloc[test_idx_df2]

In [19]:
# Train model based on the training data
model_df2 = LogisticRegression()
model_fit_df2 = model_df2.fit(X_train_df2, y_train_df2)

c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [24]:
# 預測結果為1的機率
prob_df2 = predict_for_month(model_fit_df2, df2, df2_var)
threshold_df2 = find_best_threshold(df2['direction'], prob_df2)
print(round(threshold_df2, 2)) #0.5

0.5


In [26]:
# 以閾值為界判斷預測值應為0或1
pred_df2 = predicted_labels(prob_df2, threshold_df2)
#print(pred_df2)

# 計算[無]總經數值的正確性
predicted_accuracy(df2['direction'], pred_df2) #0.64

0.64